In [23]:
# !pip3 install git+https://github.com/rongardF/tvdatafeed.git@e6f6aaa7de439ac6e454d9b26d2760ded8dc4923
from tvDatafeed import TvDatafeed, Interval

In [24]:
from helpers_db import get_engine, run_sql
engine = get_engine()

In [25]:
# queries
query_today = """SELECT a.symbol as symbol, a.exchange as exchange
                 FROM assets a INNER JOIN positions p ON a.symbol = p.symbol
                 WHERE p.quote_last_quote_at < CURRENT_DATE AND a.category='stock'"""
query_last_hour = """SELECT a.symbol, a.exchange, DATE_TRUNC('hour', now()) - quote_last_quote_at AS datediff
                     FROM assets a INNER JOIN positions p ON a.symbol = p.symbol
                     WHERE quote_last_quote_at < (DATE_TRUNC('hour', now()) - '1 hour'::interval)"""
                    #  WHERE a.category = 'stock'
assets = None
with engine.begin() as conn:
  #  SELECT symbol, exchange FROM assets WHERE category='stock'
  assets = run_sql(conn, query_last_hour).mappings().all()
# securities['symbol']
assets

[{'symbol': 'DBK', 'exchange': 'XETR', 'datediff': datetime.timedelta(seconds=14400)},
 {'symbol': 'LQQ', 'exchange': 'EURONEXT', 'datediff': datetime.timedelta(days=638, seconds=43200)},
 {'symbol': 'ON', 'exchange': 'NASDAQ', 'datediff': datetime.timedelta(seconds=7200)}]

In [26]:
from helpers_db import insert_on_conflict_nothing_ticks, truncate_table

tv = TvDatafeed()

def fetch_ticks(symbol, exchange, bars=100):
  df = tv.get_hist(
      symbol=symbol,
      exchange=exchange,
      interval=Interval.in_1_hour,
      n_bars=bars,
      extended_session=True,
  )
  return df

def fix_symbol(eachange_symbol):
  return eachange_symbol.split(':')[-1] if ':' in eachange_symbol else eachange_symbol

def normalize_dataframe(df):
  df.reset_index(inplace=True)
  df['symbol'] = df['symbol'].apply(fix_symbol)
  if 'datetime' in list(df.columns.values):
    df['dt'] = df['datetime']
    df.drop(columns=['datetime'], inplace=True)
  return df

# with engine.begin() as conn:
#   truncate_table(conn, 'ticks')

for asset in assets:
  symbol    = asset['symbol']
  exchange  = asset['exchange']
  print(f"Processing {exchange}:{symbol}")
  try:
    df = fetch_ticks(symbol, exchange, bars=7*24) # 7*24
    print(f"  count: {df.shape[0]}")
    df = normalize_dataframe(df)
    df.to_sql('ticks', engine, if_exists='append', index=False, method=insert_on_conflict_nothing_ticks)
  except Exception as e:
    print(f"Exception {exchange}:{symbol}")
    print(str(e))


Processing XETR:DBK
  count: 168
Processing EURONEXT:LQQ
  count: 168
Processing NASDAQ:ON
  count: 168
